In [ ]:
from dask.optimization_ast import compile_ast
import pickle
import dask.array as da
import numpy as np

def da_compile_ast(a):
    dsk = compile_ast(a.__dask_graph__(), a.__dask_keys__())
    return da.Array(dsk, a.name, a.chunks, a.dtype, a.shape)

def define_std():
    a = da.arange(4, chunks=2)
    a = ((a + 1) * ((a > 2) + 2))
    a = (a * 3) + np.array([7, 8, 9, 10])
    a = a.sum()
    return a

def define_ast():
    a = da.arange(4, chunks=2)
    a = ((a + 1) * ((a > 2) + 2))
    a = da_compile_ast(a)
    a = (a * 3) + np.array([7, 8, 9, 10])
    a = a.sum()
    a = da_compile_ast(a)
    return a

da_std = define_std()
da_ast = define_ast()
da_std.compute(), da_ast.compute()

In [ ]:
pickle.loads(pickle.dumps(da_ast)).compute()

In [ ]:
%timeit define_std()
%timeit da_std.compute(optimize_graph=False)
%timeit da_std.compute(optimize_graph=True)
%timeit define_ast()
%timeit da_ast.compute(optimize_graph=False)
%timeit da_ast.compute(optimize_graph=True)

In [ ]:
for k, v in da_ast.dask.items():
    print(k)
    print(v)
    print(v[0].constant_kwargs)
    print("-----------------------")
    print(v[0].source)
    print("=======================")    

In [ ]:
da_std.visualize()

In [ ]:
da_ast.visualize()